In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import metrics, datasets, preprocessing, model_selection, svm, naive_bayes, neighbors, ensemble
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, precision_recall_curve,f1_score, roc_auc_score, roc_curve, plot_roc_curve
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
!pip install xgboost
from xgboost import XGBClassifier
import xgboost as xgb

In [2]:
file = 'C:/Users/EN/DataScienceAcademy/Project5/creditcard.csv'
df=pd.read_csv(file)

In [ ]:
df.head()

In [3]:
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [7]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']
df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)
df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [5]:
##Removing Outliers
#V2
v2_fraud = df['V2'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v2_fraud, 25), np.percentile(v2_fraud, 75)
v2_iqr = q75 - q25
v2_cut_off = v2_iqr * 1.5
v2_lower, v2_upper = q25 - v2_cut_off, q75 + v2_cut_off
outliers = [x for x in v2_fraud if x < v2_lower or x > v2_upper]
df = df.drop(df[(df['V2'] > v2_upper) | (df['V2'] < v2_lower)].index)

##V3
v3_fraud = df['V3'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v3_fraud, 25), np.percentile(v3_fraud, 75)
v3_iqr = q75 - q25
v3_cut_off = v3_iqr * 1.5
v3_lower, v3_upper = q25 - v3_cut_off, q75 + v3_cut_off
outliers = [x for x in v3_fraud if x < v3_lower or x > v3_upper]
df = df.drop(df[(df['V3'] > v3_upper) | (df['V3'] < v3_lower)].index)

##V5
v5_fraud = df['V5'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v5_fraud, 25), np.percentile(v5_fraud, 75)
v5_iqr = q75 - q25
v5_cut_off = v5_iqr * 1.5
v5_lower, v5_upper = q25 - v5_cut_off, q75 + v5_cut_off
outliers = [x for x in v5_fraud if x < v5_lower or x > v5_upper]
df = df.drop(df[(df['V5'] > v5_upper) | (df['V5'] < v5_lower)].index)

##V11
v11_fraud = df['V11'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v11_fraud, 25), np.percentile(v11_fraud, 75)
v11_iqr = q75 - q25
v11_cut_off = v11_iqr * 1.5
v11_lower, v11_upper = q25 - v11_cut_off, q75 + v11_cut_off
outliers = [x for x in v11_fraud if x < v11_lower or x > v11_upper]
df = df.drop(df[(df['V11'] > v11_upper) | (df['V11'] < v11_lower)].index)

##V15
v15_fraud = df['V15'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v15_fraud, 25), np.percentile(v15_fraud, 75)
v15_iqr = q75 - q25
v15_cut_off = v15_iqr * 1.5
v15_lower, v15_upper = q25 - v15_cut_off, q75 + v15_cut_off
outliers = [x for x in v15_fraud if x < v15_lower or x > v15_upper]
df = df.drop(df[(df['V15'] > v15_upper) | (df['V15'] < v15_lower)].index)

##V25
v25_fraud = df['V25'].loc[df['Class'] == 1].values
q25, q75 = np.percentile(v25_fraud, 25), np.percentile(v25_fraud, 75)
v25_iqr = q75 - q25
v25_cut_off = v25_iqr * 1.5
v25_lower, v25_upper = q25 - v25_cut_off, q75 + v25_cut_off
outliers = [x for x in v25_fraud if x < v25_lower or x > v25_upper]
df = df.drop(df[(df['V25'] > v25_upper) | (df['V25'] < v25_lower)].index)

In [7]:
label_encoder = preprocessing.LabelEncoder() #label encoding on class_label before logistic regression
df['Class']= label_encoder.fit_transform(df['Class']) 
df['Class'].unique()

array([0, 1], dtype=int64)

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
from collections import Counter
Counter(y_adasyn)

In [ ]:
from imblearn.over_sampling import ADASYN
X_adasyn, y_adasyn = ADASYN(random_state=42).fit_sample(X,y)

In [8]:
X_adasyn_train, X_adasyn_test, y_adasyn_train, y_adasyn_test = train_test_split(X_adasyn, y_adasyn, train_size=0.8, random_state=42)

In [9]:
#Logistic regression classifier
LR = LogisticRegression(solver= 'liblinear', fit_intercept = True)
results_LR = LR.fit(X_adasyn_train, y_adasyn_train)
y_pred = LR.predict(X_adasyn_test)
print('Train accuracy:', LR.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', LR.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred), 
                                                                recall_score(y_adasyn_test, y_pred), f1_score(y_adasyn_test, y_pred)))
CV_scores_LR = cross_val_score(LR,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_LR))

Train accuracy: 0.9991874159828253
Test accuracy: 0.9992626663389628
Precision:0.8763,   Recall:0.6250, F1:0.7296
CV Accuracy 0.9991187014731682


In [ ]:
SVM = svm.SVC(kernel="linear")
results_SVM = SVM.fit(X_adasyn_train, y_adasyn_train)
y_pred_SVM = SVM.predict(X_adasyn_test)
print('Train accuracy:', SVM.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', SVM.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f}, Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_SVM), 
                                                                recall_score(y_adasyn_test, y_pred_SVM), f1_score(y_adasyn_test, y_pred_SVM)))
#CV_scores_SVM = cross_val_score(SVM,  X_adasyn, y_adasyn, cv=10)
#print('CV Accuracy', statistics.mean(CV_scores_SVM))

In [ ]:
#Gradient Boosting Classifier
GB = GradientBoostingClassifier()
results_GB = GB.fit(X_adasyn_train, y_adasyn_train)
y_pred_GB = GB.predict(X_adasyn_test)
print('Train accuracy:', GB.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', GB.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_GB), 
                                                                recall_score(y_adasyn_test, y_pred_GB), f1_score(y_adasyn_test, y_pred_GB)))
CV_scores_GB = cross_val_score(GB, X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_GB))

In [ ]:
#Confusion matrix
cm = confusion_matrix(y_adasyn_test, y_pred_GB)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, linewidths=0.5, cmap = 'Blues_r', fmt='d')
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Gradient Boosting Classification')
ax.xaxis.set_ticklabels(['No-Fraud', 'Fraud'])
ax.yaxis.set_ticklabels(['No-Fraud', 'Fraud']);

In [ ]:
#Bagging Decision Tree Classification
kfold = model_selection.KFold(n_splits=10, random_state=7)
cart = DecisionTreeClassifier()
BG = BaggingClassifier(base_estimator=cart, n_estimators=100, random_state=7)
results_BG = model_selection.cross_val_score(BG, X_adasyn_train, y_adasyn_train, cv=kfold)
BG.fit(X_adasyn_train, y_adasyn_train)
y_pred_BG = BG.predict(X_adasyn_test)
print('Train accuracy:', BG.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', BG.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_BG), 
                                                                recall_score(y_adasyn_test, y_pred_BG), f1_score(y_adasyn_test, y_pred_BG)))
CV_scores_BG = cross_val_score(BG, X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_BG))

In [ ]:
# Random Forest Classification
kfold = model_selection.KFold(n_splits=10, random_state=7)
RF = RandomForestClassifier(n_estimators=100, max_features=3)
results_RF = model_selection.cross_val_score(RF, X_adasyn_train, y_adasyn_train, cv=kfold)
RF.fit(X_adasyn_train, y_adasyn_train)
y_pred_RF = RF.predict(X_adasyn_test)
print('Train accuracy:', RF.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', RF.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_RF), 
                                                                recall_score(y_adasyn_test, y_pred_RF), f1_score(y_adasyn_test, y_pred_RF)))
CV_scores_RF = cross_val_score(RF,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_RF))

In [ ]:
# Extra Trees Classification
kfold = model_selection.KFold(n_splits=10, random_state=7)
ET = ExtraTreesClassifier(n_estimators=100, max_features=3)
results_ET = model_selection.cross_val_score(ET, X_adasyn_train, y_adasyn_train, cv=kfold)
ET.fit(X_adasyn_train, y_adasyn_train)
y_pred_ET = ET.predict(X_adasyn_test)
print('Train accuracy:', ET.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', ET.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f}, Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_ET), 
                                                                recall_score(y_adasyn_test, y_pred_ET), f1_score(y_adasyn_test, y_pred_ET)))
CV_scores_ET = cross_val_score(ET,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_ET))

In [ ]:
# AdaBoost Classification
kfold = model_selection.KFold(n_splits=10, random_state=7)
Ada = AdaBoostClassifier(n_estimators=100, random_state=7)
results_Ada = model_selection.cross_val_score(Ada, X_adasyn_train, y_adasyn_train, cv=kfold)
Ada.fit(X_adasyn_train, y_adasyn_train)
y_pred_Ada = Ada.predict(X_adasyn_test)
print('Train accuracy:', Ada.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', Ada.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f}, Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_Ada), 
                                                                recall_score(y_adasyn_test, y_pred_Ada), f1_score(y_adasyn_test, y_pred_Ada)))
CV_scores_Ada = cross_val_score(Ada,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_Ada))

In [ ]:
#Stochastic Gradient Boosting
kfold = model_selection.KFold(n_splits=10, random_state=7)
SGB = GradientBoostingClassifier(n_estimators=100, random_state=7)
results_SGB = model_selection.cross_val_score(SGB, X_adasyn_train, y_adasyn_train, cv=kfold)
SGB.fit(X_adasyn_train, y_adasyn_train)
y_pred_SGB = SGB.predict(X_adasyn_test)
print('Train accuracy:', SGB.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', SGB.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_SGB), 
                                                                recall_score(y_adasyn_test, y_pred_SGB), f1_score(y_adasyn_test, y_pred_SGB)))
CV_scores_SGB = cross_val_score(SGB,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_SGB))

In [ ]:
# Voting Ensemble for Classification
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
kfold = model_selection.KFold(n_splits=10, random_state=7)
VC = VotingClassifier(estimators)
results_VC = model_selection.cross_val_score(VC, X_adasyn_train, y_adasyn_train, cv=kfold)
VC.fit(X_adasyn_train, y_adasyn_train)
y_pred_VC = VC.predict(X_adasyn_test)
print('Train accuracy:', VC.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', VC.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_VC), 
                                                                recall_score(y_adasyn_test, y_pred_VC), f1_score(y_adasyn_test, y_pred_VC)))
CV_scores_VC = cross_val_score(VC,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_VC))

In [ ]:
XGB=xgb.XGBClassifier(random_state=7,learning_rate=0.01)
XGB.fit(X_adasyn_train, y_adasyn_train)
XGB.fit(X_adasyn_train, y_adasyn_train)
y_pred_XGB = XGB.predict(X_adasyn_test)
print('Train accuracy:', XGB.score(X_adasyn_train, y_adasyn_train))
print('Test accuracy:', XGB.score(X_adasyn_test, y_adasyn_test))
print('Precision:{:6.4f},   Recall:{:6.4f}, F1:{:6.4f}' .format(precision_score(y_adasyn_test, y_pred_XGB), 
                                                                recall_score(y_adasyn_test, y_pred_XGB), f1_score(y_adasyn_test, y_pred_XGB)))
CV_scores_XGB = cross_val_score(XGB,  X_adasyn, y_adasyn, cv=10)
print('CV Accuracy', statistics.mean(CV_scores_XGB))

In [ ]:
classifiers = [LR, GB, BG, RF, ET, Ada, XGB]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_adasyn_test, y_adasyn_test, ax=ax)

In [ ]:
data = [['LR', statistics.mean(CV_scores_LR),f1_score(y_adasyn_test, y_pred)],
    ['GB', statistics.mean(CV_scores_GB),f1_score(y_adasyn_test, y_pred_GB) ],['BG', statistics.mean(CV_scores_BG), f1_score(y_adasyn_test, y_pred_BG)], 
        ['RF', statistics.mean(CV_scores_RF), f1_score(y_adasyn_test, y_pred_RF)],['ET', statistics.mean(CV_scores_ET), f1_score(y_adasyn_test, y_pred_ET)], 
        ['Ada', statistics.mean(CV_scores_Ada), f1_score(y_adasyn_test, y_pred_Ada)], ['XGB', statistics.mean(CV_scores_XGB), f1_score(y_adasyn_test, y_pred_XGB)]] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Model', 'CV_Accuracy', 'F1']) 

In [ ]:
plt.plot( 'Model', 'CV_Accuracy', data=df, marker='', color='blue', linewidth=2)
plt.plot( 'Model', 'F1', data=df, marker='', color='red', linewidth=2, linestyle='dashed', label="F1")
plt.legend()